In [ ]:
import csv
import sys
import datetime
import threading
import numpy as np
import serial


def handle_errors(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except serial.SerialException as e:
            print(f"Could not open serial port: {e}")
            sys.exit(1)
        except IOError as e:
            print(f"Could not write to file: {e}")
            sys.exit(1)
    return wrapper


@handle_errors
def read_from_port1(ser1, captureTime, shared_data, lock):
    T=0
    while T < captureTime:
        if ser1.inWaiting() > 0:
            line = ser1.readline().decode('utf-8').strip()  # read a '\n' terminated line

            if 'BME680 test' in line:  # if 'BME680 test' is in the line
                continue  # if it is, skip this iteration of the loop

            data = line.split(',')  # split the data by ','
            
            # Get current timestamp and format it
            timestamp = datetime.datetime.now()
            #formatted_timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
            formatted_timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]


            # Add timestamp to the beginning of the data list
            data.insert(0, formatted_timestamp)

            with lock:
                shared_data['port1'] = data
            T = T + 1

@handle_errors
def read_from_port2(ser2, captureTime, shared_data, lock):
    SensData=np.zeros((captureTime,1))
    T=0
    while T < captureTime:
        a = ser2.read(32)
        for i in range(1):
            SensData[T,i] = int.from_bytes(a[0:3],'little')

        # Prepare data for writing to CSV
        data = list(SensData[T,:])

        with lock:
            shared_data['port2'] = data
        T = T + 1

try:
    ser1 = serial.Serial('COM9', 115200)  # replace 'COM9' with the port Arduino is using
    ser2 = serial.Serial('COM6', 57600)  # replace 'COM6' with the port Arduino is using

    captureTime = 180
    shared_data = {'port1': None, 'port2': None}
    lock = threading.Lock()
    with open('data.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        thread1 = threading.Thread(target=read_from_port1, args=(ser1, captureTime, shared_data, lock))
        thread2 = threading.Thread(target=read_from_port2, args=(ser2, captureTime, shared_data, lock))
        thread1.start()
        thread2.start()

        while True:
            with lock:
                if shared_data['port1'] is not None and shared_data['port2'] is not None:
                    # If both threads have written their data, write the data to the CSV file
                    writer.writerow(shared_data['port1'] + shared_data['port2'])
                    shared_data['port1'] = None
                    shared_data['port2'] = None

            if not thread1.is_alive() and not thread2.is_alive():
                # If both threads have finished, break the loop
                break

        thread1.join()
        thread2.join()

except serial.SerialException as e:
    print(f"Could not open serial port: {e}")
    sys.exit(1)

except IOError as e:
    print(f"Could not write to file: {e}")
    sys.exit(1)